# **Generate twitter**

In [0]:
pip install twitter

     |████████████████████████████████| 61kB 2.3MB/s 


In [0]:
import twitter

In [0]:
consumer_key = "3LYLKEKUNKoP3iMdADftTRV9A"
consumer_secret = "Dk4uBXfaysEflN8Tz8fXm4GIHebHCWjB7s6Ustqx41ftUrjluv"
access_token = "1235956152493641728-2reyP6mCanQEJLQW5juDS17E4aGww3"
access_token_secret = "Rj7gDBONyyCHK2bIZVUs0ijjto92wlFj8sCtCnHHukO4Q"
authorization = twitter.OAuth(access_token, access_token_secret, consumer_key, consumer_secret)

In [0]:
#import os
#output_filename = os.path.join(os.path.expanduser("~"), "data", "datasets", "twitter", "python_tweets.json")
#print(output_filename)

In [0]:
output_filename = "/content/python_tweets.json"
print(output_filename)

/content/python_tweets.json


In [0]:
#print(output_filename)

In [0]:
t = twitter.Twitter(auth=authorization)

In [0]:
import json

n_output = 0

with open(output_filename, 'a') as output_file:
    search_results = t.search.tweets(q="python", count=100)['statuses']
    for tweet in search_results:
        if 'text' in tweet:
            output_file.write(json.dumps(tweet))
            output_file.write("\n\n")
            n_output += 1

print("Saved {} entries".format(n_output))


Saved 100 entries


In [0]:
import pandas as pd
fd = pd.read_json("/content/python_tweets.json", lines=True)
print("fd")

ValueError: ignored

# **Label twitter**

In [0]:
import json
input_filename = "/content/python_tweets.json"
labels_filename = "/content/python_classes.json"

In [0]:
tweets = []
with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0:
            continue
        tweets.append(json.loads(line))

In [0]:
import os

In [0]:
labels = []
if os.path.exists(labels_filename):
  with open(labels_filename) as inf:
      labels = json.load(inf)

In [0]:
def get_next_tweet():
    return tweets[len(labels)]['text']

In [0]:
%%html
<div name="tweetbox">
    Instructions: Click in text box. Enter a 1 if the tweet is relevant, enter 0 otherwise.<br>
    Tweet: <div id="tweet_text" value="text"></div><br>
    <input type=text id="capture"></input><br>
</div>

<script>
function set_label(label){
    var kernel = IPython.notebook.kernel;
    kernel.execute("labels.append(" + label + ")");
    load_next_tweet();
}

function load_next_tweet(){
   var code_input = "get_next_tweet()";
   var kernel = IPython.notebook.kernel;
   var callbacks = { 'iopub' : {'output' : handle_output}};
   kernel.execute(code_input, callbacks, {silent:false});
}

function handle_output(out){
   console.log(out);
   var res = out.content.data["text/plain"];
   $("div#tweet_text").html(res);
}

$("input#capture").keypress(function(e) {
    console.log(e);
      if(e.which == 48) {
        // 0 pressed
        set_label(0);
        $("input#capture").val("");
      }else if (e.which == 49){
        // 1 pressed
        set_label(1);  
        $("input#capture").val("");
      }
});

load_next_tweet();
</script>

In [0]:
#with open(labels_filename, 'w') as outf:
#    json.dump(labels, outf)

# **Create replicable dataset**

In [0]:
import json
import os
input_filename = "/content/python_tweets.json"
labels_filename = "/content/python_classes.json"
replicable_dataset = "/content/replicable_dataset.json"

In [0]:
tweets = []
with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0:
            continue
        tweets.append(json.loads(line))
if os.path.exists(labels_filename):
    with open(labels_filename) as inf:
          labels = json.load(inf)

In [0]:
dataset = [(tweet['id'], label) for label, tweet in zip(labels, tweets)]

In [0]:
len(dataset), len(tweets), len(labels)

(99, 100, 99)

In [0]:
with open(replicable_dataset, 'w') as outf:
    json.dump(dataset, outf)

# **Collect tweets**

In [0]:
import json
import os
tweet_filename = "/content/replicable_python_tweets.json"
labels_filename = "/content/replicable_python_classes.json"
replicable_dataset = "/content/replicable_dataset.json"

In [0]:
with open(replicable_dataset) as inf:
    tweet_ids = json.load(inf)

In [0]:
actual_labels = []
label_mapping = dict(tweet_ids)

In [0]:
import twitter
consumer_key = "3LYLKEKUNKoP3iMdADftTRV9A"
consumer_secret = "Dk4uBXfaysEflN8Tz8fXm4GIHebHCWjB7s6Ustqx41ftUrjluv"
access_token = "1235956152493641728-2reyP6mCanQEJLQW5juDS17E4aGww3"
access_token_secret = "Rj7gDBONyyCHK2bIZVUs0ijjto92wlFj8sCtCnHHukO4Q"
authorization = twitter.OAuth(access_token, access_token_secret, consumer_key, consumer_secret)

In [0]:
all_ids = [tweet_id for tweet_id, label in tweet_ids]

with open(tweet_filename, 'a') as output_file:
    # We can lookup 100 tweets at a time, which saves time in asking twitter for them
    for start_index in range(0, len(all_ids), 100):
        #100----900
        id_string = ",".join(str(i) for i in all_ids[start_index:start_index+100])
        #100----900
        search_results = t.statuses.lookup(_id=id_string)
        for tweet in search_results:
            if 'text' in tweet:
                # Valid tweet - save to file
                output_file.write(json.dumps(tweet))
                output_file.write("\n\n")
                actual_labels.append(label_mapping[tweet['id']])

In [0]:
with open(labels_filename, 'w') as outf:
    json.dump(actual_labels, outf)

In [0]:
len(actual_labels), len(all_ids)

(198, 99)

# **Classify**

In [0]:
 s = """Three Rings for the Elven-kings under the sky, Seven for the Dwarf-lords in halls of stone, Nine for Mortal Men, doomed to die, One for the Dark Lord on his dark throne In the Land of Mordor where the Shadows lie. One Ring to rule them all, One Ring to find them, One Ring to bring them all and in the darkness bind them. In the Land of Mordor where the Shadows lie. """.lower()
words = s.split()
from collections import Counter
c = Counter(words)
print(c.most_common(5))

[('the', 9), ('for', 4), ('in', 4), ('to', 4), ('one', 4)]


In [0]:
pip install spacy

In [0]:
import spacy
from sklearn.base import TransformerMixin

# Create a spaCy parser
nlp = spacy.load('en')


class BagOfWords(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        results = []
        for document in X:
            row = {}
            for word in list(nlp(document, tag=False, parse=False, entity=False)):
                if len(word.text.strip()):
                    row[word.text] = True
            results.append(row)
        return results

In [0]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB

In [0]:
input_filename = "/content/python_tweets.json"
labels_filename = "/content/python_classes.json"
#replicable_dataset = "/content/replicable_dataset.json"
#tweet_filename = "/content/replicable_python_tweets.json"
#labels_filename = "/content/replicable_python_classses.json"
#replicable_dataset = "/content/replicable_dataset.json"

In [0]:
import json

tweets = []
with open(input_filename) as inf:
    for line in inf:
        if len(line.strip()) == 0: 
            continue
        tweets.append(json.loads(line)['text'])

with open(labels_filename) as inf:
    labels = json.load(inf)

# Ensure only classified tweets are loaded
tweets = tweets[:len(labels)]
assert len(tweets) == len(labels)

In [0]:
len(dataset), len(tweets), len(labels)

(99, 99, 99)

In [0]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('bag-of-words', BagOfWords()), ('vectorizer', DictVectorizer()), ('naive-bayes', BernoulliNB()) ])

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, tweets, labels, scoring='f1')
#We then print out the average of the scores:
import numpy as np
print("Score: {:.3f}".format(np.mean(scores)))

In [0]:
model = pipeline.fit(tweets, labels)

In [0]:
nb = model.named_steps['naive-bayes']
feature_probabilities = nb.feature_log_prob_

In [0]:
top_features = np.argsort(-nb.feature_log_prob_[1])[:50]

In [0]:
dv = model.named_steps['vectorizer']

In [0]:
for i, feature_index in enumerate(top_features):
    print(i, dv.feature_names_[feature_index], np.exp(feature_probabilities[1][feature_index]))